In [1]:
import pandas as pd
from datetime import datetime
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI
from energy_price_pred.energypricepred import *
import json
from market.ml_logic.optimsier_model_var_in_deep import *

# from market.models import price_model
# from market.ml_logic.cons_model import *
# from market.ml_logic.energy_price_model import *
# from market.ml_logic.gen_model_efficient import *

import matplotlib.pylab as plt

/home/rahuliyer/.pyenv/versions/3.11.6/envs/market/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-30 10:59:14.745052: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-30 10:59:14.745380: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-30 10:59:14.747624: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-30 10:59:14.775535: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available

In [12]:
app = FastAPI()

@app.get("/predict")
def predict(battery_size: int, # 5 total size
            battery_charge: int): # 1 initial charge amount
    """predicting Buy/Sell price as first pass"""

    # date=pd.Timestamp(date)
    output_keys = ['SalePrice_p/kwh', 'PurchasePrice_p/kwh', 'Generation_kwh', 'Consumption_kwh']

    res = run_full_model_api(int(battery_size), int(battery_charge))

    res_pred_saleprice = pd.DataFrame.from_dict(res['predicted_data']['SalePrice_p/kwh']) #pd.DataFrame.from_dict(res['predicted_data'])
    res_pred_all= pd.DataFrame.from_dict(res['predicted_data'])#pd.DataFrame.from_dict(res['predicted_data'])
    res_opt_batt= pd.DataFrame(res['optimised_battery_storage'])
    res_opt_buyprice= pd.DataFrame(res['optimised_energy_purchase_price'])
    res_opt_sellprice= pd.DataFrame(res['optimised_energy_sold_price'])
    res_opt_baseprice= pd.DataFrame(res['baseline_hourly_price'])
    res_weather_code = res['weather_code']
    res_weather_code_reset = res_weather_code.reset_index()
    res_weather_code_reset = res_weather_code_reset.drop(columns=['ds'])
    res_weather_code_reset = res_weather_code_reset['weather_code'].tolist()

    output = {'prediction_data': res_pred_all}#, 'res_opt_batt': res_opt_batt, 'res_opt_buyprice': res_opt_buyprice,
              #'res_opt_sellprice': res_opt_sellprice, 'res_opt_baseprice': res_opt_baseprice}#,
              #'res_weather_code': res_weather_code_reset}
    return output# {'keys': str(res.keys())}

@app.get("/")
def root():
    return {'greeting': 'Hello there'}

In [13]:
def predict_test(battery_size: int, # 5 total size
            battery_charge: int): # 1 initial charge amount
    """predicting Buy/Sell price as first pass"""

    # date=pd.Timestamp(date)
    output_keys = ['SalePrice_p/kwh', 'PurchasePrice_p/kwh', 'Generation_kwh', 'Consumption_kwh']

    res = run_full_model_api(int(battery_size), int(battery_charge))

    res_pred_saleprice = pd.DataFrame.from_dict(res['predicted_data']['SalePrice_p/kwh']) #pd.DataFrame.from_dict(res['predicted_data'])
    res_pred_all= pd.DataFrame.from_dict(res['predicted_data'])#pd.DataFrame.from_dict(res['predicted_data'])
    res_opt_batt= pd.DataFrame(res['optimised_battery_storage'])
    res_opt_buyprice= pd.DataFrame(res['optimised_energy_purchase_price'])
    res_opt_sellprice= pd.DataFrame(res['optimised_energy_sold_price'])
    res_opt_baseprice= pd.DataFrame(res['baseline_hourly_price'])
    res_weather_code = res['weather_code']
    res_weather_code_reset = res_weather_code.reset_index()
    res_weather_code_reset = res_weather_code_reset.drop(columns=['ds'])
    res_weather_code_reset = res_weather_code_reset['weather_code'].tolist()

    output = {'prediction_data': res_pred_all}#, 'res_opt_batt': res_opt_batt, 'res_opt_buyprice': res_opt_buyprice,
              #'res_opt_sellprice': res_opt_sellprice, 'res_opt_baseprice': res_opt_baseprice}#,
              #'res_weather_code': res_weather_code_reset}
    return output# {'keys': str(res.keys())}

In [14]:
output = predict_test(5, 1)
output

Cost forecasted
Energy consumption forecasted
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Model optimsied
Basline model running


{'prediction_data':                      SalePrice_p/kwh  PurchasePrice_p/kwh  Generation_kwh  \
 ds                                                                          
 2024-03-30 00:00:00         7.440950            14.881899             0.0   
 2024-03-30 01:00:00         7.008441            14.016881             0.0   
 2024-03-30 02:00:00         6.531323            13.062646             0.0   
 2024-03-30 03:00:00         6.409842            12.819684             0.0   
 2024-03-30 04:00:00         6.760686            13.521372             0.0   
 ...                              ...                  ...             ...   
 2024-04-01 19:00:00        10.475176            20.950352             0.0   
 2024-04-01 20:00:00         8.893433            17.786867             0.0   
 2024-04-01 21:00:00         7.869465            15.738931             0.0   
 2024-04-01 22:00:00         7.607336            15.214672             0.0   
 2024-04-01 23:00:00         7.709580        

In [17]:
output['prediction_data']

,SalePrice_p/kwh,PurchasePrice_p/kwh,Generation_kwh,Consumption_kwh
ds,,,,
2024-03-30 00:00:00,7.440950,14.881899,0.0,0.429404
2024-03-30 01:00:00,7.008441,14.016881,0.0,0.365603
2024-03-30 02:00:00,6.531323,13.062646,0.0,0.317523
2024-03-30 03:00:00,6.409842,12.819684,0.0,0.286281
2024-03-30 04:00:00,6.760686,13.521372,0.0,0.279110
...,...,...,...,...
2024-04-01 19:00:00,10.475176,20.950352,0.0,0.716893
2024-04-01 20:00:00,8.893433,17.786867,0.0,0.731578
2024-04-01 21:00:00,7.869465,15.738931,0.0,0.688024


In [ ]:
!pwd

In [15]:
df = output['prediction_data']
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72 entries, 2024-03-30 00:00:00 to 2024-04-01 23:00:00
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   SalePrice_p/kwh      72 non-null     float64
 1   PurchasePrice_p/kwh  72 non-null     float64
 2   Generation_kwh       72 non-null     float32
 3   Consumption_kwh      72 non-null     float64
dtypes: float32(1), float64(3)
memory usage: 2.5 KB


In [ ]:
plt.plot(df.index, df['weather_code'])

In [ ]:
output['prediction_data'].info()

In [ ]:
df_reset = df.reset_index()
df_reset.head()

In [ ]:
output['res_weather_code']['weather_code']

In [ ]:
df_reset_drop = df_reset.drop(columns=['ds'])
df_reset_drop['weather_code'].astype(int)

In [ ]:
df_reset_drop.info()

In [ ]:
df_reset_drop